In [1]:
from pathlib import Path
from time import sleep 

import numpy as np

import roboticstoolbox as rtb
import spatialmath as sm

from manipylator import VisualRobot
from manipylator import utils

# Load Robot
Below we load a URDF file describing Manny the robot manipulator. A URDF is a textual representation of a robot, defining the parameters we need so we can calculate the forward & reverse kinemtics. 

In [2]:
# path = Path('/workspace/robots/robot.urdf')
path = Path('/workspace/robots/measured/robot.urdf')
manny = VisualRobot(path)
print(manny.model)

[I 05/20/25 01:52:59.336 271] [shell.py:_shell_pop_print@23] Graphical python shell detected, using wrapped sys.stdout
[Genesis] [01:53:05] [INFO] ╭───────────────────────────────────────────────╮
[Genesis] [01:53:05] [INFO] │┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈ Genesis ┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈│
[Genesis] [01:53:05] [INFO] ╰───────────────────────────────────────────────╯
[Genesis] [01:53:06] [INFO] Running on [NVIDIA GeForce MX250] with backend gs.cuda. Device memory: 1.95 GB.
[Genesis] [01:53:06] [INFO] 🚀 Genesis initialized. 🔖 version: 0.2.1, 🌱 seed: None, 📏 precision: '32', 🐛 debug: False, 🎨 theme: 'dark'.
[Genesis] [01:53:07] [INFO] Scene <f731856> created.
[Genesis] [01:53:07] [INFO] Adding <gs.RigidEntity>. idx: 0, uid: <359cf74>, morph: <gs.morphs.Plane>, material: <gs.materials.Rigid>.
[Genesis] [01:53:07] [INFO] Adding <gs.RigidEntity>. idx: 1, uid: <9d4258a>, morph: <gs.morphs.URDF(file='/workspace/robots/measured/robot.urdf')>, material: <gs.materials.Rigid>.
[Genesis] [01:53:10] [INFO] Building

ERobot: measured, 6 joints (RRRRRR), dynamics, geometry, collision
┌──────┬──────────────────┬───────┬──────────────────┬─────────────────────────────────────────────────────────────────────────────┐
│ link │       link       │ joint │      parent      │                             ETS: parent to link                             │
├──────┼──────────────────┼───────┼──────────────────┼─────────────────────────────────────────────────────────────────────────────┤
│    0 │ base             │       │ BASE             │ SE3()                                                                       │
│    1 │ carriage_1       │     0 │ base             │ SE3(-7.47e-06, -2.937e-06, 0.037; -180°, -7.914e-41°, -6.361e-15°) ⊕ Rz(q0) │
│    2 │ shoulder_lift    │     1 │ carriage_1       │ SE3(-0.065, -2.469e-06, -0.055; 180°, 90°, 180°) ⊕ Rz(q1)                   │
│    3 │ elbow            │     2 │ shoulder_lift    │ SE3(0.355, -2.338e-06, -4.032e-06; -180°, -3.45e-15°, 3.069e-12°) ⊕ Rz(q2)  │
│ 

# Visualizing Manny
Using the `manny.visualizer.robot` object we can get the properties of it's links. For example, a given link's location in 3D space. Below we find the location of the end effector in when the all joint parameters are set to 0 degrees:

In [3]:
robot = manny.visualizer.robot
# robot

In [4]:
ee = robot.get_link('end_effector')
# ee

# Tracing a Path in R3
Below we calculate a positions in R3 that correspond to a heart shaped curve:

In [48]:
scale = 0.05
x_offset = -0.0885
y_offset = 0.48
height = 0.09

# -0.0885,  0.2954,  0.2595

shape = utils.parametric_heart_1(np.linspace(0, 2*np.pi, 30))
positions = scale * shape
positions[:,0] += x_offset
positions[:,1] += y_offset
positions[:,2] = height

positions[:3]

array([[-0.0885    ,  0.48      ,  0.09      ],
       [-0.08779754,  0.48339824,  0.09      ],
       [-0.08326534,  0.4921947 ,  0.09      ]])

Each row is the (x, y, z) coordinates of a point on the curve. This is useful if we'd like to visualize the expected path of the end effector.

In [49]:
# for x, y, z in positions:
#     manny.visualizer.scene.draw_debug_sphere([x,y,z])
manny.visualizer.scene.clear_debug_objects()
manny.visualizer.scene.draw_debug_spheres(positions)



In [50]:
robot.set_dofs_position([0, 0.2, 2.35, 1, 1.57, 0])

res = robot.inverse_kinematics(
        link = ee,
        pos  = [-0.0885, 0.48, 0.09],
        quat = manual_quat,
        ) 
res

tensor([-1.5286e-04, -7.4387e-01,  1.8333e+00,  1.1116e+00,  1.5729e+00,
         1.6756e-03], device='cuda:0')

Similarly, we can calculate the joint parameters of each position:

In [51]:
robot.set_dofs_position(res)

manny.visualizer.scene.step()
ee.get_pos()

tensor([-0.0885,  0.4800,  0.0900], device='cuda:0')

In [52]:
manual_quat = ee.get_quat()
manual_quat

tensor([ 9.9862e-01,  5.2587e-02, -1.0848e-03,  8.5592e-04], device='cuda:0')

In [53]:
quat_pointing_down = [0, 1, 0, 0]

qs = [robot.inverse_kinematics(
        link = ee,
        pos  = pos,
        quat = manual_quat,
        ) for pos in positions]
qs[:5]

[tensor([-1.5315e-04, -7.4391e-01,  1.8333e+00,  1.1116e+00,  1.5729e+00,
          1.6756e-03], device='cuda:0'),
 tensor([ 1.3248e-03, -7.4895e-01,  1.8198e+00,  1.1032e+00,  1.5727e+00,
          3.1452e-03], device='cuda:0'),
 tensor([ 0.0109, -0.7613,  1.7872,  1.0830,  1.5717,  0.0126], device='cuda:0'),
 tensor([ 0.0314, -0.7762,  1.7492,  1.0598,  1.5696,  0.0330], device='cuda:0'),
 tensor([ 0.0620, -0.7854,  1.7256,  1.0454,  1.5664,  0.0635], device='cuda:0')]

In [54]:
import pandas as pd
df = pd.DataFrame(qs, columns=['q1', 'q2', 'q3', 'q4', 'q5', 'q6']).astype(float)
df.head()
df.to_csv('heart_qs.csv', index=False, float_format='%.3f')

And to visualize the poses Manny would take, we can animate the path:

In [55]:
for q in qs:
    robot.set_dofs_position(q)
    # print(robot.get_dofs_position())
    manny.visualizer.scene.step()
    sleep(0.1)


In [56]:
robot.get_dofs_position()

tensor([-1.5343e-04, -7.4395e-01,  1.8333e+00,  1.1116e+00,  1.5729e+00,
         1.6756e-03], device='cuda:0')

In [41]:
robot.set_dofs_position([0, 0.2, 2.35, 1, 1.57, 0])

manny.visualizer.scene.step()
ee.get_pos()

tensor([-0.0885,  0.2572,  0.2888], device='cuda:0')

In [13]:
a = robot.inverse_kinematics(
        link = ee,
        pos  = [-0.0885,  0.2954,  0.2595],
        quat = quat_pointing_down,
        )
a

tensor([ 5.3207e-05, -1.3284e-01,  2.3897e+00,  9.5175e-01, -1.5708e+00,
        -3.1416e+00], device='cuda:0')

In [14]:
robot.set_dofs_position(a)

manny.visualizer.scene.step()
ee.get_pos()

tensor([-0.0885,  0.2954,  0.2595], device='cuda:0')